In [1]:
using TensorDecomposition, LinearAlgebra, Combinatorics, InvertedIndices, HomotopyContinuation

In [2]:
function rlim(n, d)
    if iseven(d)
         if n == 2
             return binomial(Int(n+d/2), n) - n
         else
             return binomial(Int(n+d/2), n) - n - 1
         end
     else 
         return binomial(Int(n+(d-1)/2), n)
     end
 end;

function cofactor(A::AbstractMatrix)
    ax = axes(A)
    out = similar(A, eltype(A), ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[Not(col), Not(row)])
        end
    end
    return out
end;

function alpha_iterator(::Val{N}, s, t=()) where {N}
    N <= 1 && return ((s, t...),) # Iterator over a single Tuple
    Iterators.flatten(alpha_iterator(Val(N-1), s-i, (i, t...)) for i in 0:s)
end

function monomialOrder(a, b)
    if a[1] > b[1]
        return true
    elseif a[1] < b[1]
        return false
    else 
        if maximum(a[2:end]) < maximum(b[2:end])
            return true
        elseif maximum(a[2:end]) > maximum(b[2:end])
            return false
        else
            return !isless(a[2:end], b[2:end])
        end 
    end
end

function convertIndices(x)
    d = sum(x)
    y = zeros(Int, d)
    c = 1
    for i=1:length(x)
        pow = x[i]
        y[c:c+pow-1] .= i
        c += pow
    end 
    return y .- 1
end;

function e(j, n)
    e = zeros(Int64, n)
    e[j] = 1
    return e
end;

function multBasis(basis, j, n)
    ej = e(j, n)
    ej[1] = -1
    return [Tuple(Int(y) for y in [x for x in b] + ej) for b in basis]
end

multMonomial(b, ej) = Tuple(Int(y) for y in [x for x in b] + ej);

function khatri_rao(A, d)
    if d == 1
        B = A
    else
        n, r = size(A)
        B = zeros(eltype(A), (n^d, r))
        for i=1:r 
            B[:, i] = kron(ntuple(x->A[:, i], d)...)
        end;
    end;
    return B
end

function krDrop(A, d)
    n = size(A)[1]-1
    inds = collect(with_replacement_combinations(1:n+1, d))
    inds = map(x->from_multiindex(x, n+1), inds)
    return khatri_rao(A, d)[inds, :]
end;

function from_multiindex(x, n)
    d = length(x)
    c = 0
    for i=1:d-1
        c += (x[i]-1)*n^(d-i)
    end
    return c + x[d]
end;

### Real tensor

In [398]:
n = 3
d = 4;
rlim(n, d)

6

In [399]:
r = n+2

5

In [400]:
T, A_, L = randomRankedTensor(n+1, d, r, real=true);

In [401]:
Thank = hankMat(T);

In [402]:
monomials = reverse(sort(collect(alpha_iterator(Val(n+1), Int(d)))));
D = Dict()
for (i, mon) in enumerate(monomials)
   D[mon] = i 
end

basis = monomials[1:r]

5-element Vector{NTuple{4, Int64}}:
 (4, 0, 0, 0)
 (3, 1, 0, 0)
 (3, 0, 1, 0)
 (3, 0, 0, 1)
 (2, 2, 0, 0)

In [403]:
first_r = [D[b] for b in basis];

In [404]:
H0 = Float64.(Thank[first_r, first_r]);

In [405]:
H0_adj = cofactor(H0);
H0_det = det(H0);
H0_inv = inv(H0);

In [406]:
Hs = []
for i=2:n+1
    col_inds = [D[b] for b in multBasis(basis, i, n+1)]
    push!(Hs, Thank[first_r, col_inds])
end

In [407]:
vars = unique(reduce(vcat, ([reduce(vcat, HomotopyContinuation.variables.(H)) for H in Hs])))

3-element Vector{Variable}:
 h₃₆
 h₃₇
 h₃₈

In [408]:
# linearEquations = []

# for i=1:n
#     for j=i+1:n
#         append!(linearEquations, HomotopyContinuation.expand.(Hs[i][n+2:end, :]*H0_adj*Hs[j][:, 2:n+1]-Hs[j][n+2:end, :]*H0_adj*Hs[i][:, 2:n+1]))
#     end
# end

In [409]:
# i = 1
# j = 2

# HomotopyContinuation.expand.(Hs[i][n+2:end, :]*H0_inv*Hs[j][:, 2:n+1]-Hs[j][n+2:end, :]*H0_inv*Hs[i][:, 2:n+1])

In [410]:
# alpha_i = multBasis(basis, i+1, n+1)[n+2]
# alpha_j = multBasis(basis, j+1, n+1)[n+2]

# beta_i = multBasis(basis, i+1, n+1)[3]
# beta_j = multBasis(basis, j+1, n+1)[3]

# Gij = Thank[vcat(first_r, D[alpha_i]), vcat(first_r, D[beta_j])]
# Gji = Thank[vcat(first_r, D[alpha_j]), vcat(first_r, D[beta_i])]

# HomotopyContinuation.expand.((det(Gij)-det(Gji))/H0_det) 

In [411]:
# HomotopyContinuation.expand.(permutedims(Thank[first_r, D[alpha_i]])*H0_inv*Thank[first_r, D[beta_j]]-permutedims(Thank[first_r, D[alpha_j]])*H0_inv*Thank[first_r, D[beta_i]])

In [412]:
function process_2sidelinEq(a_i, b_j, a_j, b_i, D, H0_inv, Thank, base)
    return HomotopyContinuation.expand.(permutedims(Thank[base, D[a_i]])*H0_inv*Thank[base, D[b_j]]-permutedims(Thank[base, D[a_j]])*H0_inv*Thank[base, D[b_i]])
end;

function process_1sidelinEq(a_i, b_j, D, H0_inv, Thank, base)
    return HomotopyContinuation.expand.(permutedims(Thank[base, D[a_i]])*H0_inv*Thank[base, D[b_j]].+ (-1)^length(base) * Thank[D[a_i], D[b_j]])
end;

In [413]:
linearEquations2 = []
zero_2side_eqs = Set()
zero_1side_eqs = Set()
nonzero_2side_eqs = Set()

for alpha_ in basis[n+2:end]
    alpha = [a_ for a_ in alpha_]
    for beta_ in basis[2:n+1]
        beta = [b_ for b_ in beta_]
        for i=1:n
            ei = e(i+1, n+1)
            ei[1] = -1
            for j=i+1:n
                ej = e(j+1, n+1)
                ej[1] = -1
                
                a_i = Tuple(alpha+ei)
                a_j = Tuple(alpha+ej)
                b_i = Tuple(beta+ei)
                b_j = Tuple(beta+ej)
                
                if b_i in basis && b_j in basis
                    push!(zero_2side_eqs, ((a_i, b_j), (a_j, b_i)))
                elseif b_i in basis
                    push!(zero_1side_eqs, (a_i, b_j))
                elseif b_j in basis
                    push!(zero_1side_eqs, (a_j, b_i))
                    
                else
                    push!(nonzero_2side_eqs, ((a_i, b_j), (a_j, b_i)))
                end
                
#                 append!(linearEquations2, process_linEq(a_i, b_j, a_i, b_j, D, H0_inv, Thank, first_r))
                
            end
        end
    end
end

In [414]:
zero_1side_eqs = sort([eq for eq in zero_1side_eqs])

2-element Vector{Tuple{NTuple{4, Int64}, NTuple{4, Int64}}}:
 ((1, 3, 0, 0), (2, 1, 0, 1))
 ((1, 3, 0, 0), (2, 1, 1, 0))

In [415]:
nonzero_2side_eqs = sort([eq for eq in nonzero_2side_eqs])

7-element Vector{Tuple{Tuple{NTuple{4, Int64}, NTuple{4, Int64}}, Tuple{NTuple{4, Int64}, NTuple{4, Int64}}}}:
 (((1, 2, 1, 0), (2, 0, 0, 2)), ((1, 2, 0, 1), (2, 0, 1, 1)))
 (((1, 2, 1, 0), (2, 0, 1, 1)), ((1, 2, 0, 1), (2, 0, 2, 0)))
 (((1, 2, 1, 0), (2, 1, 0, 1)), ((1, 2, 0, 1), (2, 1, 1, 0)))
 (((1, 3, 0, 0), (2, 0, 0, 2)), ((1, 2, 0, 1), (2, 1, 0, 1)))
 (((1, 3, 0, 0), (2, 0, 1, 1)), ((1, 2, 0, 1), (2, 1, 1, 0)))
 (((1, 3, 0, 0), (2, 0, 1, 1)), ((1, 2, 1, 0), (2, 1, 0, 1)))
 (((1, 3, 0, 0), (2, 0, 2, 0)), ((1, 2, 1, 0), (2, 1, 1, 0)))

In [425]:
alleqs = vcat(nonzero_2side_eqs, zero_1side_eqs)

9-element Vector{Tuple{Tuple{Any, Any, Vararg{Int64}}, Tuple{Any, Any, Vararg{Int64}}}}:
 (((1, 2, 1, 0), (2, 0, 0, 2)), ((1, 2, 0, 1), (2, 0, 1, 1)))
 (((1, 2, 1, 0), (2, 0, 1, 1)), ((1, 2, 0, 1), (2, 0, 2, 0)))
 (((1, 2, 1, 0), (2, 1, 0, 1)), ((1, 2, 0, 1), (2, 1, 1, 0)))
 (((1, 3, 0, 0), (2, 0, 0, 2)), ((1, 2, 0, 1), (2, 1, 0, 1)))
 (((1, 3, 0, 0), (2, 0, 1, 1)), ((1, 2, 0, 1), (2, 1, 1, 0)))
 (((1, 3, 0, 0), (2, 0, 1, 1)), ((1, 2, 1, 0), (2, 1, 0, 1)))
 (((1, 3, 0, 0), (2, 0, 2, 0)), ((1, 2, 1, 0), (2, 1, 1, 0)))
 ((1, 3, 0, 0), (2, 1, 0, 1))
 ((1, 3, 0, 0), (2, 1, 1, 0))

In [417]:
A1 = []
b1 = []
for tup in zero_1side_eqs
    eq = process_1sidelinEq(tup[1], tup[2], D, H0_inv, Thank, first_r)[1]
    coeffs = HomotopyContinuation.coeffs_as_dense_poly(eq, vars, 1)
    push!(A1, coeffs[1:end-1])
    push!(b1, -coeffs[end])
end

A2 = []
b2 = []
for tup in nonzero_2side_eqs
    eq = process_2sidelinEq(tup[1][1], tup[1][2], tup[2][1], tup[2][2], D, H0_inv, Thank, first_r)[1]
    coeffs = HomotopyContinuation.coeffs_as_dense_poly(eq, vars, 1)
    push!(A2, coeffs[1:end-1])
    push!(b2, -coeffs[end])
end

A1 = reduce(hcat,A1)';
b1 = Float64.(b1);

A2 = reduce(hcat,A2)';
b2 = Float64.(b2);

A = vcat(A2, A1);

In [418]:
A1

2×3 adjoint(::Matrix{Float64}) with eltype Float64:
 -1.11095   0.0  -1.0
 -1.36959  -1.0   0.0

In [419]:
A2

7×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.0        0.489933  -0.94497
 0.0        0.94497   -1.30112
 0.0       -1.11095    1.36959
 0.489933   0.0        1.11095
 0.94497    0.0        1.36959
 0.94497    1.11095    0.0
 1.30112    1.36959    0.0

In [422]:
 A

9×3 Matrix{Float64}:
  0.0        0.489933  -0.94497
  0.0        0.94497   -1.30112
  0.0       -1.11095    1.36959
  0.489933   0.0        1.11095
  0.94497    0.0        1.36959
  0.94497    1.11095    0.0
  1.30112    1.36959    0.0
 -1.11095    0.0       -1.0
 -1.36959   -1.0        0.0

In [423]:
A[[9, 7, 6, 8, 5, 4, 3, 2, 1], :]

9×3 Matrix{Float64}:
 -1.36959   -1.0        0.0
  1.30112    1.36959    0.0
  0.94497    1.11095    0.0
 -1.11095    0.0       -1.0
  0.94497    0.0        1.36959
  0.489933   0.0        1.11095
  0.0       -1.11095    1.36959
  0.0        0.94497   -1.30112
  0.0        0.489933  -0.94497

In [426]:
alleqs[[9, 7, 6, 8, 5, 4, 3, 2, 1]]

9-element Vector{Tuple{Tuple{Any, Any, Vararg{Int64}}, Tuple{Any, Any, Vararg{Int64}}}}:
 ((1, 3, 0, 0), (2, 1, 1, 0))
 (((1, 3, 0, 0), (2, 0, 2, 0)), ((1, 2, 1, 0), (2, 1, 1, 0)))
 (((1, 3, 0, 0), (2, 0, 1, 1)), ((1, 2, 1, 0), (2, 1, 0, 1)))
 ((1, 3, 0, 0), (2, 1, 0, 1))
 (((1, 3, 0, 0), (2, 0, 1, 1)), ((1, 2, 0, 1), (2, 1, 1, 0)))
 (((1, 3, 0, 0), (2, 0, 0, 2)), ((1, 2, 0, 1), (2, 1, 0, 1)))
 (((1, 2, 1, 0), (2, 1, 0, 1)), ((1, 2, 0, 1), (2, 1, 1, 0)))
 (((1, 2, 1, 0), (2, 0, 1, 1)), ((1, 2, 0, 1), (2, 0, 2, 0)))
 (((1, 2, 1, 0), (2, 0, 0, 2)), ((1, 2, 0, 1), (2, 0, 1, 1)))

In [175]:
# A = []
# b = []
# for eq in linearEquations2
#     coeffs = HomotopyContinuation.coeffs_as_dense_poly(eq, vars, 1)
#     push!(A, coeffs[1:end-1])
#     push!(b, -coeffs[end])
# end
# A = reduce(hcat,A)';
# b = Float64.(b);
# # A = map!(x -> isapprox(x, 0, atol=1e-10) ? 0 : x, A, A);
# # A = round.(A, digits=10);
# # A = unique(A, dims=1);

# A

In [176]:
# A[4, :]-A[2, :]

In [177]:
# sol = A\b;

# Ms = []
# for H in Hs
#     push!(Ms, HomotopyContinuation.evaluate.(H*H0_inv, vars => sol))
# end;